In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob

In [29]:
test_gts = sorted(glob(os.path.join('/scratch/project/bollmann_lab/MedSAM_Laptop/datasets/test_demo/gts', '*')))
print(len(test_gts))
test_gts = np.load(test_gts[-1],allow_pickle=True)
print(list(test_gts.keys()))

10
['gts', 'spacing']


In [11]:
val_gts = sorted(glob(os.path.join('/scratch/project/bollmann_lab/MedSAM_Laptop/datasets/validation/litemedsam-seg-val-imgs', '*')))
print(len(val_gts))

3076


In [20]:
test_gts = np.load(val_gts[-1],allow_pickle=True)
print(list(test_gts.keys()))
print(test_gts['segs'].shape)

['segs']
(227, 192, 192)


In [4]:
val_imgs = sorted(glob(os.path.join('/scratch/project/bollmann_lab/MedSAM_Laptop/datasets/test_demo/imgs', '*')))
print(len(val_imgs))
test_imgs = np.load(val_imgs[-1],allow_pickle=True)
print(list(test_imgs.keys()))
print(test_imgs['imgs'].shape, test_imgs['boxes'].shape)
np.mean(test_imgs['imgs']), np.min(test_imgs['imgs']), np.max(test_imgs['imgs'])

10
['imgs', 'spacing', 'boxes']
(74, 240, 240) (1, 6)


(41.10066136448949, 0, 255)

In [24]:
test_imgs['spacing']

array([3.64583325, 3.64583325, 3.26999998])

In [27]:
val_segs = sorted(glob(os.path.join('/scratch/project/bollmann_lab/MedSAM_Laptop/datasets/validation/segs', '*.npz')))
print(len(val_segs))
test_segs = np.load(val_segs[-1],allow_pickle=True)
print(list(test_segs.keys()))
# print(test_segs['segs'].shape, test_segs['boxes'].shape)

10
['segs']


In [51]:
from evaluation.SurfaceDice import compute_surface_distances, compute_surface_dice_at_tolerance, compute_dice_coefficient
from os.path import join
compute_NSD = True
def compute_metrics(npz_name):
    metric_dict = {'dsc': -1.}
    if compute_NSD:
        metric_dict['nsd'] = -1.
    npz_img = np.load(join(img_dir, npz_name), allow_pickle=True, mmap_mode='r')
    npz_seg = np.load(join(seg_dir, npz_name), allow_pickle=True, mmap_mode='r')
    npz_gt = np.load(join(gt_dir, npz_name), allow_pickle=True, mmap_mode='r')
    gts = npz_gt['segs']
    segs = npz_seg['segs']
    if npz_name.startswith('3D'):
        spacing = npz_img['spacing']
    
    dsc = compute_multi_class_dsc(gts, segs)
    # comupute nsd
    if compute_NSD:
        if dsc > 0.2:
        # only compute nsd when dice > 0.2 because NSD is also low when dice is too low
            if npz_name.startswith('3D'):
                nsd = compute_multi_class_nsd(gts, segs, spacing)
            else:
                spacing = [1.0, 1.0, 1.0]
                nsd = compute_multi_class_nsd(np.expand_dims(gts, -1), np.expand_dims(segs, -1), spacing)
        else:
            nsd = 0.0
    return npz_name, dsc, nsd
    
def compute_multi_class_dsc(gt, seg):
    dsc = []
    for i in range(1, gt.max()+1):
        gt_i = gt == i
        seg_i = seg == i
        dsc.append(compute_dice_coefficient(gt_i, seg_i))
    return np.mean(dsc)

def compute_multi_class_nsd(gt, seg, spacing, tolerance=2.0):
    nsd = []
    for i in range(1, gt.max()+1):
        gt_i = gt == i
        seg_i = seg == i
        surface_distance = compute_surface_distances(
            gt_i, seg_i, spacing_mm=spacing
        )
        nsd.append(compute_surface_dice_at_tolerance(surface_distance, tolerance))
    return np.mean(nsd)


In [57]:
img_dir = '/scratch/project/bollmann_lab/MedSAM_Laptop/datasets/validation/imgs'
seg_dir = '/scratch/project/bollmann_lab/MedSAM_Laptop/datasets/validation/segs'
gt_dir = '/scratch/project/bollmann_lab/MedSAM_Laptop/datasets/validation/litemedsam-seg-val-imgs'
npz_names = sorted(os.listdir(gt_dir))
npz_names = [npz_name for npz_name in npz_names if npz_name.endswith('.npz')]
name, dsc, nsd = compute_metrics(npz_names[-6])

In [58]:
name, dsc, nsd

('3DBox_MR_0619.npz', 0.9996548153261995, 1.0)